<a href="https://colab.research.google.com/github/mingd00/menu-translation/blob/main/menu_image_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 메뉴판 이미지에서 텍스트를 추출하는 모델 생성

## 0. 라이브러리 설치 및 불러오기

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import glob
import shutil
import json
from PIL import Image
import yaml

In [ ]:
!pip install ultralytics
!pip install -U albumentations

## 1. 경로 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/menu_project'

## 2. 데이터 불러오기 및 yolo_dataset 폴더 생성

In [ ]:
data = os.path.join(path + '/data/meta_data.zip')
menu_info_file =  os.path.join(path + '/data/menu_info.json')

print(data)
print(menu_info_file)

/content/drive/MyDrive/menu_project/data/meta_data.zip
/content/drive/MyDrive/menu_project/data/menu_info.json


* 데이터를 옮길 폴더 설정

In [ ]:
# base_folder에 image, label 폴더 생성
base_folder = '/content/yolo_dataset'
image_folder = os.path.join(base_folder, 'images')
label_folder = os.path.join(base_folder, 'labels')

# 각각 train과 val 폴더 생성
os.makedirs(os.path.join(image_folder, 'train'), exist_ok=True)
os.makedirs(os.path.join(image_folder, 'val'), exist_ok=True)
os.makedirs(os.path.join(label_folder, 'train'), exist_ok=True)
os.makedirs(os.path.join(label_folder, 'val'), exist_ok=True)

# 특정 폴더 삭제
'''
# 삭제하려는 폴더 경로
folders_to_delete = [
    '/content/yolo_dataset',
]

# 폴더 삭제
for folder in folders_to_delete:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted: {folder}")
    else:
        print(f"Folder not found: {folder}")
'''

# 폴더 경로 설정
image_train_path = os.path.join(image_folder, 'train')
image_valid_path = os.path.join(image_folder, 'val')
label_train_path = os.path.join(label_folder, 'train')
label_valid_path = os.path.join(label_folder, 'val')

## 3. 데이터 처리(yolo 학습 구조로 파일 정리)



### 1) 이미지 처리 함수: 폴더 안에 zip 파일 열어서 풀기

In [ ]:
# zip 파일 풀어서 파일 옮기기
def move_img_files(target_folder, tmp_folder, dst_path):
  for target in target_folder:
    zip_ref.extract(target, tmp_folder) # zip 파일을 임시 폴더로 추출
    extracted_zip_path = os.path.join(tmp_folder, target) # 추출된 zip 파일 경로

    # 내부 zip 파일 열기
    with zipfile.ZipFile(extracted_zip_path, 'r') as inner_zip:
      inner_zip.extractall(tmp_folder) # 모든 파일 추출
      # 추출된 파일 yolo_dataset 폴더로 이동
      for img_file in inner_zip.namelist():
        if img_file.endswith('.jpg'):
          img_src = os.path.join(tmp_folder, img_file)
          img_dst = os.path.join(dst_path, os.path.basename(img_file))
          shutil.move(img_src, img_dst)
      print(f"Moved: {extracted_zip_path}")

### 2) 라벨 처리 함수: zip파일 열고 json -> txt로 변환 후 옮기기

In [ ]:
# json -> txt 파일로 형식 맞춰서 변환
# menu_data를 딕셔너리로 변환 (ko를 키로 사용)
with open(menu_info_file, 'r') as f:
      menu_data = json.load(f)
menu_dict = {menu['ko']: menu['class'] for menu in menu_data}

def json2txt(json_path, json_data, yolo_path):
  json_data = json.load(json_data) # 입력 json 파일 읽기

  # 사진 전체 가로, 세로 길이
  image_width = json_data.get("meta", {}).get("image_original_width")
  image_height = json_data.get("meta", {}).get("image_original_height")

  yolo_lines = [] # 라벨 데이터를 모을 리스트
  for annotation in json_data.get("annotations", []):
    ocr_info = annotation.get("ocr", {})
    x = ocr_info.get("x")
    y = ocr_info.get("y")
    width = ocr_info.get("width")
    height = ocr_info.get("height")

    # YOLO 형식의 데이터로 변환
    class_id = menu_dict.get(annotation.get("menu_information", {}).get("ko")) # 클래스 아이디

    x_center = (x + width / 2) / image_width
    y_center = (y + height / 2) / image_height
    width_ratio = width / image_width
    height_ratio = height / image_height

    # YOLO 형식으로 포맷
    yolo_format = f"{class_id} {x_center} {y_center} {width_ratio} {height_ratio}"
    yolo_lines.append(yolo_format)

  # YOLO 폴더에 txt 데이터 옮기기
  txt_file_name = os.path.splitext(os.path.basename(json_path))[0] + '-tf' + '.txt'
  yolo_path = os.path.join(yolo_path, txt_file_name)
  with open(yolo_path, 'w', encoding='utf-8') as f:
          f.write("\n".join(yolo_lines))

# zip 파일 풀어서 파일 옮기기
def move_label_files(target_folder, tmp_folder, dst_path):
  for target in target_folder:
    zip_ref.extract(target, tmp_folder) # zip 파일을 임시 폴더로 추출
    extracted_zip_path = os.path.join(tmp_folder, target) # 추출된 zip 파일 경로

    # 내부 zip 파일 열기
    with zipfile.ZipFile(extracted_zip_path, 'r') as inner_zip:
      # 추출된 파일 json -> txt 변환
      for label_file in inner_zip.namelist():
        if label_file.endswith('.json'):
          json_path = os.path.join(tmp_folder, label_file)
          json_file = inner_zip.open(label_file)
          json2txt(json_path, json_file, dst_path)

      print(f"Moved: {extracted_zip_path}")

### 3) 전체 파일 압축 해제 후 이동

In [ ]:
# 임시 폴더 (압축 해제용)
tmp = '/content/tmp'
os.makedirs(tmp, exist_ok=True)

# 압축 파일 열기
with zipfile.ZipFile(data, 'r') as zip_ref:
  # 이미지 파일 선택
  train_image = [f for f in zip_ref.namelist() if 'images/train/' in f and f.endswith('.zip')]
  valid_image = [f for f in zip_ref.namelist() if 'images/val/' in f and f.endswith('.zip')]
  train_label = [f for f in zip_ref.namelist() if 'labels/train/' in f and f.endswith('.zip')]
  valid_label = [f for f in zip_ref.namelist() if 'labels/val/' in f and f.endswith('.zip')]

  # 'train', 'valid' 이미지 파일
  print(f"Train images: {train_image}, Valid images: {valid_image}")
  move_img_files(train_image, tmp, image_train_path)
  move_img_files(valid_image, tmp, image_valid_path)

  # 'train', 'valid' 라벨 파일들 출력
  print(f"Train labels: {train_label}, Valid labels: {valid_label}")
  move_label_files(train_label, tmp, label_train_path)
  move_label_files(valid_label, tmp, label_valid_path)

Train images: ['images/train/busan.zip', 'images/train/seoul.zip'], Valid images: ['images/val/jeolla.zip']
Moved: /content/tmp/images/train/busan.zip
Moved: /content/tmp/images/train/seoul.zip
Moved: /content/tmp/images/val/jeolla.zip
Train labels: ['labels/train/busan.zip', 'labels/train/seoul.zip'], Valid labels: ['labels/val/jeolla.zip']
Moved: /content/tmp/labels/train/busan.zip
Moved: /content/tmp/labels/train/seoul.zip
Moved: /content/tmp/labels/val/jeolla.zip


In [ ]:
# 각 요소의 개수 확인
print(len(os.listdir(image_train_path)))
print(len(os.listdir(image_valid_path)))
print(len(os.listdir(label_train_path)))
print(len(os.listdir(label_valid_path)))

365
47
365
47


In [ ]:
# 이미지, 라벨 파일 이름 일치 여부 확인
def check_dataset_consistency(image_dir, label_dir):
    image_files = {os.path.splitext(f)[0] for f in os.listdir(image_dir) if f.endswith('.jpg')}
    label_files = {os.path.splitext(f)[0] for f in os.listdir(label_dir) if f.endswith('.txt')}

    missing_labels = image_files - label_files
    missing_images = label_files - image_files

    if missing_labels:
        print("Missing label files for images:", sorted(list(missing_labels)))
        print(len(missing_labels))
    if missing_images:
        print("Missing image files for labels:", sorted(list(missing_images)))
        print(len(missing_images))

    if not missing_labels and not missing_images:
        print("All files are consistent!")

check_dataset_consistency(image_train_path, label_train_path)

All files are consistent!


### 4) yaml 파일 작성

In [ ]:
# menu_info JSON 파일에서 {클래스: 메뉴명} 추출
with open(menu_info_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# class_names 딕셔너리 생성
class_names = {item["class"]-1: item["ko"] for item in data}
print(class_names)

# 데이터셋 구조 정의
dataset_structure = {
    'path': '/content/yolo_dataset',  # 데이터셋 루트 경로
    'train': 'images/train',           # 훈련 이미지 경로
    'val': 'images/val',               # 검증 이미지 경로
    "nc": len(class_names),            # 클래스 개수
    'names': class_names               # 클래스 이름
}

# YAML 파일로 저장
yaml_file_path = '/content/dataset_structure.yaml'
with open(yaml_file_path, 'w', encoding='utf-8') as yaml_file:
    yaml.dump(dataset_structure, yaml_file, default_flow_style=False, allow_unicode=True)

print(f"YAML 파일이 생성되었습니다.")

{0: '간장마늘치킨', 1: '후라이드치킨', 2: '떡강정', 3: '양념치친', 4: '치드뿌치텐더', 5: '후라이드텐더', 6: '테라생맥주', 7: '오렌지주스', 8: '탄산음료', 9: '하이네켄', 10: '맘스스낵볼세트', 11: '치즈뿌치감자(체다)', 12: '코울슬로', 13: '치즈스틱', 14: '핫크리스피버거', 15: '리아미라클버거', 16: '클래식치즈버거', 17: '새우버거', 18: '불고기버거', 19: '티렉스버거', 20: '크레이지핫팩', 21: '치킨매니아팩', 22: '충분해팩', 23: '베이스볼버거팩', 24: '시그니처투게더팩', 25: '어메이징세트', 26: '메가바이트세트', 27: '치즈베이컨세트', 28: '크런치윙', 29: '치킨시저샐러드콤보', 30: '슈퍼소닉팩', 31: '콜라', 32: '사이다', 33: '제로슈거콜라', 34: '꿔바로우', 35: '어향고기', 36: '경장육슬', 37: '샹라고기', 38: '건두부고추볶음', 39: '마파두부', 40: '궁보기정', 41: '깐벤오징어', 42: '샹라대하', 43: '마라대하', 44: '버섯청경채', 45: '어향가지', 46: '지삼선', 47: '가지튀김', 48: '수주육편', 49: '깐궈막창', 50: '깐궈오리머리', 51: '훙소육란요리', 52: '고기구이요리', 53: '홍소향어', 54: '쯥붕어요리', 55: '영빈세트', 56: '찹쌀탕수육+짜장', 57: '찹쌀탕수육+짬뽕+볶음밥', 58: '깐풍기+짜장', 59: '깐풍기+짬뽕또는볶음밥', 60: '깐쇼새우+짜장', 61: '깐쇼새우+짬뽕또는볶음밥', 62: '크림새우+짜장', 63: '크림새우+짬뽕/볶음밥', 64: '꿔바러우', 65: '양꼬치', 66: '매운양꼬치', 67: '가지볶음', 68: '생양꼬치', 69: '오돌뼈볶음', 70: '닭똥집볶음', 71: '갈비살', 72: '고급양갈비', 73: '토마토계란볶음', 74: '건두부볶음

## 4. YOLO 모델로 학습

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11n.pt", task='detect')

In [ ]:
model.train(data="/content/dataset_structure.yaml", epochs=1, imgsz=640)

Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/dataset_structure.yaml, epochs=1, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning /content/yolo_dataset/labels/train... 342 images, 0 backgrounds, 24 corrupt: 100%|██████████| 365/365 [00:11<00:00, 31.91it/s]

train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_IF02_M0003_9002445_1-tf.jpg: ignoring corrupt image/label: division by zero
train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_KF01_M0002_510820_1-tf.jpg: ignoring corrupt image/label: division by zero
train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_KF01_M0002_580795_3-tf.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_KF04_M0002_480710_1-tf.jpg: ignoring corrupt image/label: division by zero
train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_KF04_M0002_9003836_1-tf.jpg: ignoring corrupt image/label: division by zero
train: WARNING ⚠️ /content/yolo_dataset/images/train/BS_KF05_M0002_9001865_1-tf.jpg: ignoring corrupt image/label: division by zero
train: WARNING ⚠️ /content/yolo_dataset/images/train/SL_FF01_M0002_351197_1-tf.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /content/yolo_dataset/images/train/SL_FF01_M0004_48786_1-tf.jpg: corrupt JPEG restored and s

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /content/yolo_dataset/labels/val... 43 images, 0 backgrounds, 4 corrupt: 100%|██████████| 47/47 [00:05<00:00,  8.08it/s]

val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_IF02_M0001_1578419_8-tf.jpg: ignoring corrupt image/label: division by zero
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_IF03_M0001_1501041_1-tf.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_IF03_M0002_1488015_1-tf.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_KF01_M0001_1535705_6-tf.jpg: ignoring corrupt image/label: division by zero
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_KF01_M0002_1505894_1-tf.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_KF01_M0002_1522183_1-tf.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_KF03_M0001_1539722_6-tf.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR_KF03_M0001_1579396_4-tf.jpg: ignoring corrupt image/label: division by zero
val: WARNING ⚠️ /content/yolo_dataset/images/val/JR

Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=3e-06, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        1/1      15.6G          0      89.03          0          0        640: 100%|██████████| 22/22 [00:42<00:00,  1.95s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 모델 평가
eval = model.val()  # 검증 결과 반환
eval

In [ ]:
# 추론 테스트
result = model('/content/~.jpg')
result

### 5) 모델 저장

In [ ]:
model.save("model.pt")